<a href="https://colab.research.google.com/github/thesuhas/Signlate/blob/main/CNN_with_backup_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from sklearn.model_selection import train_test_split
import os

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#training_set_path = "//Users//divyashekar//Documents//archive//asl_alphabet_train//asl_alphabet_train"
#testing_set_path = "/Users/divyashekar/Documents/Signlate/archive/asl_alphabet_test/asl_alphabet_test"

In [40]:
IMAGE_SIZE = 100
num_classes = 36
epochs = 100
initializer = 'he_normal'

In [41]:
def label(folder_name):
    num = folder_name
    if '0' in num:
        return 0
    elif '1' in num:
        return 1 
    elif '2' in num:
        return 2
    elif '3' in num:
        return 3
    elif '4' in num:
        return 4
    elif '5' in num:
        return 5
    elif '6' in num:
        return 6
    elif '7' in num:
        return 7
    elif '8' in num:
        return 8
    elif '9' in num:
        return 9
    elif 'a' in num:
        return 10
    elif 'b' in num:
        return 11
    elif 'c' in num:
        return 12
    elif 'd' in num:
        return 13
    elif 'e' in num:
        return 14
    elif 'f' in num:
        return 15
    elif 'g' in num:
        return 16
    elif 'h' in num:
        return 17
    elif 'i' in num:
        return 18
    elif 'j' in num:
        return 19
    elif 'k' in num:
        return 20
    elif 'l' in num:
        return 21
    elif 'm' in num:
        return 22
    elif 'n' in num:
        return 23
    elif 'o' in num:
        return 24
    elif 'p' in num:
        return 25
    elif 'q' in num:
        return 26
    elif 'r' in num:
        return 27
    elif 's' in num:
        return 28
    elif 't' in num:
        return 29
    elif 'u' in num:
        return 30
    elif 'v' in num:
        return 31
    elif 'w' in num:
        return 32
    elif 'x' in num:
        return 33
    elif 'y' in num:
        return 34
    elif 'z' in num:
        return 35

In [42]:
smol_dataset_path = "/content/drive/MyDrive/Colab Notebooks/asl_dataset"

In [7]:
#dataset = image_dataset_from_directory(smol_dataset_path, labels='inferred', label_mode='int', image_size=[100, 100], interpolation='nearest', batch_size=64, shuffle=False)

Found 2515 files belonging to 36 classes.


CREATING DATASET

In [43]:
def create_dataset(path):
    data = []
    for folder in os.scandir(path):
        if(os.path.isdir(folder.path)):
            for image in os.scandir(os.path.join(path, folder)):
                letter_label = label(folder.name)
                if (letter_label == None):
                    print(folder.name)
                    break
                img = cv2.imread(os.path.join(path, folder, image))

                # Resize
                new = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
                data.append([new, letter_label])
    return data

In [44]:
data = create_dataset(smol_dataset_path)

In [45]:
X = np.array([d[0] for d in data])
len(X)

2515

In [46]:
X = np.divide(X, 255)
len(X)
X.shape

(2515, 100, 100, 3)

In [47]:
y = np.array([d[1] for d in data])
len(y)

2515

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state = 1, stratify=y_train)

CNN MODEL

In [51]:
model = Sequential([
    layers.InputLayer(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu', kernel_initializer=initializer),
    layers.MaxPool2D(),
    layers.BatchNormalization(),
    # Dropout layer for regularisation
    layers.Dropout(rate=0.5),
    layers.Conv2D(32, 3, padding='same', activation='relu', kernel_initializer=initializer),
    layers.MaxPool2D(),
    layers.BatchNormalization(),
    # Dropout layer for regularisation
    layers.Dropout(rate=0.5),
    layers.Conv2D(64, 5, padding='same', activation='relu', kernel_initializer=initializer),
    layers.MaxPool2D(),
    layers.BatchNormalization(),
    # Dropout layer for regularisation
    layers.Dropout(rate=0.5),
    # Flattening output to connect to Densely Connected Layers
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_initializer=initializer),
    layers.BatchNormalization(),
    layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)
]) 

In [54]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [55]:
X_train.shape

(1414, 100, 100, 3)

In [56]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 100, 100, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 50, 50, 16)        64        
_________________________________________________________________
dropout_3 (Dropout)          (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 50, 50, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 25, 25, 32)       

In [57]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)

In [58]:
history = model.fit(x=X_train, y=y_train, epochs=epochs, validation_data = (X_val, y_val), use_multiprocessing = True, batch_size=32, callbacks=[callback])

Epoch 1/100
45/45 [==============================] - 1s 16ms/step - loss: 2.6448 - accuracy: 0.3429 - val_loss: 1.9935 - val_accuracy: 0.4809
Epoch 2/100
45/45 [==============================] - 0s 10ms/step - loss: 0.7226 - accuracy: 0.7997 - val_loss: 2.6614 - val_accuracy: 0.3581
Epoch 3/100
45/45 [==============================] - 0s 10ms/step - loss: 0.4807 - accuracy: 0.8491 - val_loss: 2.5476 - val_accuracy: 0.4258
Epoch 4/100
45/45 [==============================] - 0s 10ms/step - loss: 0.3341 - accuracy: 0.9077 - val_loss: 2.6143 - val_accuracy: 0.4089
Epoch 5/100
45/45 [==============================] - 0s 10ms/step - loss: 0.3035 - accuracy: 0.9040 - val_loss: 2.7039 - val_accuracy: 0.3771
Epoch 6/100
45/45 [==============================] - 0s 10ms/step - loss: 0.2136 - accuracy: 0.9463 - val_loss: 2.4971 - val_accuracy: 0.4492
Epoch 7/100
45/45 [==============================] - 0s 10ms/step - loss: 0.1717 - accuracy: 0.9577 - val_loss: 2.1773 - val_accuracy: 0.4640
Epoch 

In [61]:
model.evaluate(X_test, y_test)

20/20 [==============================] - 0s 7ms/step - loss: 0.1427 - accuracy: 0.9634


[0.14266391098499298, 0.9634340405464172]

In [67]:
tf.keras.models.save_model()

INFO:tensorflow:Assets written to: C:\Users\krith\Desktop\Signlate\model/assets


In [65]:
max(history.history['val_accuracy'])

0.9745762944221497

In [77]:
file_path = r"./content/C:\Users\krith\Desktop\Signlate\model"

In [75]:
loaded_model = tf.keras.models.load_model(file_path)

OSError: ignored